In [1]:
from pynq import Overlay
import numpy as np
import pandas as pd
ol = Overlay("c_dense.bit")
ol.ip_dict
 

{'dense_0': {'type': 'ayush:cnn:dense:1.0',
  'mem_id': 's_axi_control',
  'memtype': 'REGISTER',
  'gpio': {},
  'interrupts': {},
  'parameters': {'C_S_AXI_CONTROL_ADDR_WIDTH': '6',
   'C_S_AXI_CONTROL_DATA_WIDTH': '32',
   'C_M_AXI_GMEM_ID_WIDTH': '1',
   'C_M_AXI_GMEM_ADDR_WIDTH': '64',
   'C_M_AXI_GMEM_DATA_WIDTH': '32',
   'C_M_AXI_GMEM_AWUSER_WIDTH': '1',
   'C_M_AXI_GMEM_ARUSER_WIDTH': '1',
   'C_M_AXI_GMEM_WUSER_WIDTH': '1',
   'C_M_AXI_GMEM_RUSER_WIDTH': '1',
   'C_M_AXI_GMEM_BUSER_WIDTH': '1',
   'C_M_AXI_GMEM_USER_VALUE': '0x00000000',
   'C_M_AXI_GMEM_PROT_VALUE': '"000"',
   'C_M_AXI_GMEM_CACHE_VALUE': '"0011"',
   'C_M_AXI_GMEM_ENABLE_ID_PORTS': 'true',
   'C_M_AXI_GMEM_ENABLE_USER_PORTS': 'false',
   'Component_Name': 'design_1_dense_0_0',
   'clk_period': '10',
   'machine': '64',
   'combinational': '0',
   'latency': '250306',
   'II': 'x',
   'EDK_IPTYPE': 'PERIPHERAL',
   'C_S_AXI_CONTROL_BASEADDR': '0x40000000',
   'C_S_AXI_CONTROL_HIGHADDR': '0x4000FFFF',
   'ADD

In [2]:
dense_ip = ol.dense_0
mmio = dense_ip.mmio

In [3]:
dense_ip.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0),
  inputs_1 = Register(inputs=write-only),
  inputs_2 = Register(inputs=write-only),
  final_1 = Register(final=write-only),
  final_2 = Register(final=write-only)
}

In [4]:
from pynq import allocate
import numpy as np 
 
a = allocate(shape=(256,), dtype=np.float32)   #input1
b = allocate(shape=(10,), dtype=np.float32)   #final_1 

In [5]:
print("py_buffer physical address {}".format(hex(a.physical_address)))
print("py_buffer physical address {}".format(hex(b.physical_address)))

py_buffer physical address 0x16849000
py_buffer physical address 0x1684a000


In [6]:
dense_ip.register_map.inputs_1 = a.physical_address 
dense_ip.register_map.final_1  = b.physical_address

## Load input 

In [8]:
import numpy as np

# Load the .npy file
c = np.load('c.npy')
c.shape

(256,)

In [10]:
a[:] = c
# Send start signal
dense_ip.register_map.CTRL.AP_START =1

# Wait until algorithm has completed
while (dense_ip.register_map.CTRL.AP_DONE == 0):
    pass

In [11]:
b

PynqBuffer([3.55265026e-20, 5.87020683e-15, 2.58276798e-12,
            3.71440414e-14, 1.29235303e-10, 4.18979905e-20,
            1.81758742e-25, 1.00000000e+00, 1.03622996e-19,
            1.11198366e-12], dtype=float32)